# HW3 Kmeans

本此作業以map-reduce實作Kmeans演算法。為了方便起見，我將(a)小題(使用Euclidean distance的code)與(b)小題(使用Manhattan distance)的code分開來。兩者的作法是一模一樣的，差別只在把兩個map function的Euclidean distance換成Manhattan distance。因此我就只在這份程式碼中解釋我的作法，另一份Kmeans_Manhattan中就只在不同的地方做說明。

## 使用的package

In [1]:
from pyspark import SparkConf, SparkContext
from scipy.spatial import distance
from scipy.linalg import norm

## Map/Reduce function

readfile()為讀資料的函式。它會讀進每一筆58維的資料$p_k$後，計算該筆資料的norm，形成$(||p_k||, p_k)$的結構。

In [2]:
def readfile(line):
    wordlist=line.split("\n")
    maplist=[]
    for item in wordlist:
        s = item.split(" ")
        data=[]
        for item in s:
            a=float(item)
            data.append(a)
        maplist.append((norm(data),data)) # add to map
    return maplist

initial_clustering_map()是讀進來後第一次進行cluster判斷的函式。先計算該數據$p_k$與所有centroid$[c_1,...,c_{10}]$的距離，使$||p_k-c_m||$有最小值者$m$即$p_k$所屬分群。輸出的格式為$$\bigg(||p_k||,\big(m, p_k, c_m, ||p_k-c_m||\big)\bigg)$$

In [3]:
def initial_clustering_map(x):
    d= []
    for center in x[2]: # run through all c1,...,c10
        dis = distance.euclidean(x[1], center)**2 # compute distance^2
        d.append(dis) # store in list
    cluster = d.index(min(d)) # get the index with least distance
    return(x[0], (cluster, x[1], x[2][cluster], min(d)))

k_means_map()是迴圈中進行cluster判斷的函式。先計算該數據$p_k$與所有centroid$[c_1,...,c_{10}]$的距離，對使$||p_k-c_m||$有最小值者$m$者，若此時||p_k-c_m||比迴圈中上一迴圈中$p_k$到所屬centroid的距離還要小，那就將$p_k$分到新的centroid，反之則不動。輸出的格式為$$\bigg(||p_k||,\big(m, p_k, c_m, ||p_k-c_m||\big)\bigg)$$

In [4]:
def k_means_map(x):
    d = []
    for center in x[2]:
        dis = distance.euclidean(x[1][1], center)**2 # compute distance^2
        d.append(dis) # store in list
    
    min_d = min(d)
    cluster = d.index(min_d) # get the index with least distance
    if (min_d < x[1][3]):
        return (x[0], (cluster, x[1][1], x[2][cluster], min_d)) # update and assign to new cluster
    else:
        return (x[0], x[1]) # remain in the same cluster

new_centroid_mapper()為計算新的centroid時會用到的function，輸入為某群所有資料的向量和，輸出為此向量和除以該群的資料數量。假設以$n(m)$代表第$m$群data數量，輸出格式為$$\bigg(m,c_m=\frac{\sum_{k,p_k\in m} p_k}{n(m)}\bigg)$$

In [5]:
def new_centroid_mapper(x):
    z=[]
    for component in x[1]: # all component of n-dimension vector
        z.append(component/num_cluster_dict[x[0]])
    return (x[0], z)

## 呼叫SparkConf

In [6]:
sc.stop()

In [7]:
conf = SparkConf().setMaster("local").setAppName("wordcount") # call sparkconf
conf = SparkConf().set("spark.default.parallelism", 4)
sc = SparkContext(conf=conf) # call sparkcontext

## Initialize

讀入資料集。

In [8]:
dataset_intial = sc.textFile("data.txt").flatMap(readfile) # read txt file [||p_k||, p_k]

讀入centroid，為了方便起見直接用讀dataset的code讀c1.txt，接著將第二行collect出來，這樣就不必再寫一個讀檔案的function了。

In [9]:
c1 = sc.textFile("c1.txt").flatMap(readfile) # read txt file
c1 = c1.map(lambda x: x[1])
c1 = c1.collect() # list of centroids [c_1, c_2, ..., c_10]

loss_function會儲存loop中每一次迭代的loss_function的值，而cluster_dict是一個字典，會儲存每次迭代中各群的數目。

In [10]:
loss_function = [] # value of loss function in each iteration
cluster_dict = [] # number of points in each cluster in each iteration

首先先將dataset給map成$$\bigg[||p_k||, p_k, [c_1, c_2, ..., c_{10}]\bigg]$$的形式，接著再經過initial_clustering_map()後完成初始的分群，這時dataset中每一筆資料的格式為$$\bigg[||p_k||, \big(m, p_k, c_m, ||p_k-c_m||\big)\bigg]$$
亦即以$||p_k||$為key，其餘數值為value。

In [11]:
dataset = dataset_intial.map(lambda x: [x[0], x[1], c1]) # [||p_k||, p_k, [c_1, c_2, ..., c_10]]
dataset = dataset.map(initial_clustering_map) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||)], m為cluster編號            
R = dataset

## Loop

每次迭代都會依序進行以下的計算，一共迭代20次。迴圈中使用的是R這個rdd，其初始值與dataset一樣為：$$\bigg[||p_k||, \big(m, p_k, c_m, ||p_k-c_m||\big)\bigg]$$每次我們都會把R拆成D與new_centroid兩個rdd。

1. 把新計算的centroid $[c_1, ..., c_{10}]$接回去，將R給map成
$$\bigg[||p_k||, \big(m, p_k, c_m, ||p_k-c_m||\big), [c_1, c_2, ..., c_{10}]\bigg]$$

2. 利用k_means_map()重新分群，R的格式恢復為$$\bigg[||p_k||, \big(m, p_k, c_m, ||p_k-c_m||\big)\bigg]$$

3. 將R拆出D這個rdd，D是以每一群(因為有10個centroid，因此共有10群)的index為key，該群中每一筆資料到該群的centroid的距離為value，即$$(m, ||p_k-c_m||)$$

4. 將D的value加總(不論key為何)，得到本次迭代的loss function的值。

5. 將R拆出另一個rdd，稱為new_centroid，其格式首先是$(m, p_k)$，即以某筆數據所在的群的index為key，該筆數據本身為value。

6. 計算第$m$群中data的數目，存到num_cluster_dict這個字典裡。

7. 將new_centroid reduce一次。計算所有相同key(同一群)的data的向量和。即$$\bigg(m, \sum_{k,p_k\in m} p_k\bigg)$$故此時new_centroid只剩下10個元素
8. 利用new_centroid_mapper()將每一筆data除以其所在群的資料數，得出新的質心。即$$\bigg(m,c_m=\frac{\sum_{k,p_k\in m} p_k}{n(m)}\bigg)$$
9. 將new_centroid collect出來，成為新的centroid的list。

In [12]:
for i in range (1,21): # 20次要設21
    
    # 將新的c接回去
    R = R.map(lambda x: [x[0], x[1], c1]) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||), [c_1, c_2, ..., c_10]]
    
    # 重新分群
    R = R.map(k_means_map) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||)]
    D = R.map(lambda x: (x[1][0], x[1][3])) # (m, ||p_k-c_m||)
    
    loss = D.values().sum() # compute loss function
    loss_function.append(loss) # add to list
    
    new_centroid = R.map(lambda x: (x[1][0], x[1][1])) # (m, p_k)
    num_cluster_dict = new_centroid.countByKey() # compute the amount of data in each cluster
    cluster_dict.append(num_cluster_dict)
    
    new_centroid = new_centroid.reduceByKey(lambda x, y: [t1+t2 for t1, t2 in zip(x, y)]) # element-wise adding
    new_centroid = new_centroid.map(new_centroid_mapper) # (m, c_m)
    c1 = new_centroid.values().collect() # list of centroids [c_1, c_2, ..., c_10]

## Case of c2.txt

與處理c_1一模一樣，只是將讀的檔案換成c_2。

In [29]:
def new_centroid_mapper2(x):
    z=[]
    for component in x[1]:
        z.append(component/num_cluster_dict2[x[0]])
    return (x[0], z)

In [27]:
c2 = sc.textFile("c2.txt").flatMap(readfile) # read txt file
c2 = c2.map(lambda x: x[1])
c2 = c2.collect() # list of centroids [c_1, c_2, ..., c_10]
loss_function2 = [] # value of loss function in each iteration
cluster_dict2 = [] # number of points in each cluster in each iteration
dataset = dataset_intial.map(lambda x: [x[0], x[1], c2]) # [||p_k||, p_k, [c_1, c_2, ..., c_10]]
dataset = dataset.map(initial_clustering_map) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||)], m為cluster編號            
R = dataset

In [30]:
for i in range (1,21): # 20次要設21
    
    # 將新的c接回去
    R = R.map(lambda x: [x[0], x[1], c2]) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||), [c_1, c_2, ..., c_10]]
    
    # 重新分群
    R = R.map(k_means_map) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||)]
    D = R.map(lambda x: (x[1][0], x[1][3])) # (m, ||p_k-c_m||)
    
    loss = D.values().sum()
    loss_function2.append(loss)
    
    new_centroid = R.map(lambda x: (x[1][0], x[1][1])) # (m, p_k)
    num_cluster_dict2 = new_centroid.countByKey()
    cluster_dict2.append(num_cluster_dict2)
    
    new_centroid = new_centroid.reduceByKey(lambda x, y: [t1+t2 for t1, t2 in zip(x, y)]) # element-wise adding
    new_centroid = new_centroid.map(new_centroid_mapper2) # (m, c_m)
    c2 = new_centroid.values().collect() # list of centroids [c_1, c_2, ..., c_10]

## 資料處理

觀察loss function的值。由於我跑程式碼的順序的問題，導致第一次迭代的loss_function2多被儲存一個...不過後面答案大概是對的，應該是沒關係。

In [15]:
print(loss_function)

[623660345.3064115, 509862908.2975454, 485480681.87200826, 463997011.68501294, 460969266.5729968, 460537847.9827684, 460313099.6535447, 460003523.8894082, 459570539.3177352, 459021103.3422909, 458490656.1919812, 457944232.5879751, 457558005.19867724, 457290136.35230196, 457050555.0595629, 456892235.6153559, 456703630.7370345, 456404203.01897514, 456177800.541994, 455986871.02734685]


In [31]:
print(loss_function2)

[438747790.02791756, 438747790.02791756, 249803933.62600276, 194494814.40631253, 169804841.45154318, 156295748.80627593, 149094208.10896596, 142508531.61961532, 132303869.40652987, 117170969.83719078, 108547377.17857003, 102237203.3179959, 98278015.74975666, 95630226.12177409, 93793314.051193, 92377131.96821065, 91541606.25423889, 91045573.83042456, 90752240.10140806, 90470170.18122731, 90216416.1756313]


將loss function寫到csv檔案中

In [52]:
import csv

# 開啟輸出的 CSV 檔案
with open('loss_function_euclidean.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(loss_function)
    writer.writerow(loss_function2)

計算每個centroid彼此間的歐式距離與曼哈頓距離，儲存在一個二維的list中。

In [61]:
eucl_of_c1 = [] # 利用歐式距離計算loss function時，歐式距離下，c1中centroid彼此的距離
for i in range(0,10):
    eucl_of_c1.append([])
    for j in range(0,10):
        eucl_of_c1[i].append(distance.euclidean(c1[i],c1[j]))

eucl_of_c2 = [] # 利用歐式距離計算loss function時，歐式距離下，c2中centroid彼此的距離
for i in range(0,10):
    eucl_of_c2.append([])
    for j in range(0,10):
        eucl_of_c2[i].append(distance.euclidean(c2[i],c2[j]))
        
manh_of_c1 = [] # 利用歐式距離計算loss function時，曼哈頓距離下，c1中centroid彼此的距離
for i in range(0,10):
    manh_of_c1.append([])
    for j in range(0,10):
        manh_of_c1[i].append(distance.cityblock(c1[i],c1[j]))

manh_of_c2 = [] # 利用歐式距離計算loss function時，曼哈頓距離下，c2中centroid彼此的距離
for i in range(0,10):
    manh_of_c2.append([])
    for j in range(0,10):
        manh_of_c2[i].append(distance.cityblock(c2[i],c2[j]))

儲存到csv中。

In [62]:
import csv

with open('eucli_c1_euclidean.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(eucl_of_c1)

with open('eucli_c2_euclidean.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(eucl_of_c2)
    
with open('eucli_c1_manh.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(manh_of_c1)

with open('eucli_c2_manh.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(manh_of_c2)